#### Файлы 1 и 2 идентичны
Созданы для ускорения обработки
##### 1 - выделение клиентских данных из data2
##### 2 - выделение данных тех. поддержки из data2

# Иерархия файлов
### Main_file1 - выделение клиентских данных из data2
### Main_file2 - выделение данных тех. поддержки из data2
### Main_file3 - выделение данных из data почты
### Main_file4 -  выделение данных из data1
### Main_file5 - Объединение всех данных
### Main_file6 - Лемматизация, токенизация, gensim модели тематического моделирования
### Main_file7 - Построение отображения двойной кластеризации, примеры идей и моделей
### Остальные файлы - примеры гипотетической работы на сервере pythoneverywhere на flask
### Описание.pdf - краткое описание реализации

In [1]:
#Импорт базовых файлов и библиотек
import pandas as pd
import numpy as np
import re

#Лемматизация
from pymystem3 import Mystem
m = Mystem()

#Исправление опечаток
#Загружаем стандартную обученную модель
import jamspell
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('/Users/alexanpolyakov/Downloads/ru_small.bin')

True

In [2]:
#Загружаем данные data2
data2 = pd.read_csv("dialogs_2.csv", sep=',')

In [208]:
#Заменяем NaN на -1 для удобства
data2 = data2.fillna({'client_id': -1, 'user_id': -1})
#Приводим столбцы к единым типам и удаляем пустые строки в тексте
data2['client_id'] = data2['client_id'].apply(int)
data2['user_id'] = data2['user_id'].apply(int)
data2 = data2.dropna(subset=['text']).reset_index(drop=True)
data2['text'] = data2['text'].apply(str)
#Оставляем только переписку без системных сообщений
data2 = data2[data2.msg_type == 0]
#Индекс с 0
data2 = data2.reset_index(drop=True)
#На данном этапе столбец типа сообщений нам больше не нужен
data2 = data2.drop('msg_type', axis=1)
#В дальнейшем столбец поддержки понадобится для выявления плохих консультантов, достанем его по номеру инцидента
data2 = data2.drop('user_id', axis=1)
data2 = data2.reset_index(drop=True)
#Сортируем данные по номеру инцидента, так как они идут в неправильной последовательности, мы же хотим объединить тексты
data2 = data2.sort_values(by='appeal_id', ascending=0)
data2 = data2.reset_index(drop=True)

In [230]:
from progressbar import ProgressBar
import time
#Выделяем тексты пользовательских обращений для последующей кластеризации
#k = 1 - клиенты
#k = -1 - тех. поддержка
def prepare_data2(data2, k):
    
    #Запоминаем номер первого инцидента
    start = data2.appeal_id[0]
    #Датафрейм пользовательских обращений
    data2_clienttexts = pd.DataFrame(columns=['appeal_id', 'client_id', 'channel_id', 'channel_name','channel_type', 'text'])
    data2_clienttexts.text= ['' for _ in range(len(data2))]
    data2_clienttexts = data2_clienttexts.fillna('')
    #index - номер строки нового датафрейма
    index=0
    #first - первый ли текст в конкретной переписке
    first = True
    for row in range(len(data2)):
        #если встретили инцидент впервые
        if data2.appeal_id[row] == start:
            if data2.client_id[row] == k:
                data2_clienttexts.iloc[index].text += ' ' + data2.iloc[row].text
        else:
            #Новый инцидент
            start = data2.appeal_id[row]
            if data2.client_id[row] == k:
                first = True
                index += 1
                data2_clienttexts.iloc[index].text += ' ' + data2.iloc[row].text
        
        if first:
            if data2.client_id[row] == k:
                #Записываем все остальные поля, кроме текста
                first = False
                data2_clienttexts.iloc[index].appeal_id = data2.iloc[row].appeal_id
                data2_clienttexts.iloc[index].client_id = data2.iloc[row].client_id
                data2_clienttexts.iloc[index].channel_id = data2.iloc[row].channel_id
                data2_clienttexts.iloc[index].channel_name = data2.iloc[row].channel_name
                data2_clienttexts.iloc[index].channel_type = data2.iloc[row].channel_type
        pbar.update(row)
        
        
    data2_clienttexts['text'].replace('', np.nan, inplace=True)
    data2_clienttexts = data2_clienttexts.dropna()
    return data2_clienttexts

In [231]:
#Применили к data2, (В котором telegram и прочее)
#k = 1 - клиенты
#k = -1 - тех. поддержка
pbar = ProgressBar(maxval=len(data2))
pbar.start()
data2_clienttexts = prepare_data2(data2, 1)
pbar.finish()

100% (2758169 of 2758169) |##############| Elapsed Time: 0:12:26 Time:  0:12:26


In [232]:
#Записываем клиентские данные и переходим в следующий notebook
#Для ускорения работы, параллельно в файле 2 обработали также данные тех.поддержки
data2_clienttexts.to_pickle('data2_clienttexts.pickle')

In [1]:
#предпредобработка :)
#Собранные клиентские данные с data2
#data2_clienttexts

In [2]:
#data2_clienttexts[data2_clienttexts.client_id != -1]